# Loading Fake Timeseries Surface Data

This notebook is designed to explore some functionality with loading DataFiles and using Loaders.

This example will require some extra optional libraries, including nibabel and nilearn! Note: while nilearn is not imported, when trying to import SingleConnectivityMeasure, if nilearn is not installed, this will give an ImportError.

We will also use fake data for this example - so no special datasets required!

In [1]:
import BPt as bp
import nibabel as nib
import numpy as np
import pandas as pd
import os

In [2]:
def save_fake_timeseries_data():
    '''Save fake timeseries and fake surface data.'''
    
    X = np.random.random(size = (20, 100, 10242))
    os.makedirs('fake_time_data', exist_ok=True)
    
    for x in range(len(X)):
        np.save('fake_time_data/' + str(x) + '_lh', X[x])
    for x in range(len(X)):
        np.save('fake_time_data/' + str(x) + '_rh', X[x])
        
save_fake_timeseries_data()

In [3]:
# Init a Dataset
data = bp.Dataset()

Next, we are interested in loading in the files to the dataset as data files. There are a few different ways to do this, but we will use the method add_data_files. We will try and load the timeseries data first.

First we need a dictionary mapping desired column name to location or a file glob (which is easier so let's use that).

In [4]:
# The *'s just mean wildcard
files = {'timeseries_lh': 'fake_time_data/*_lh*',
         'timeseries_rh': 'fake_time_data/*_rh*'}

# Now let's try loading with 'auto' as the file to subject function
data.add_data_files(files, 'auto')

,timeseries_lh,timeseries_rh
13,Loc(0),Loc(36)
9,Loc(1),Loc(27)
8,Loc(2),Loc(37)
2,Loc(3),Loc(35)
16,Loc(4),Loc(38)
11,Loc(5),Loc(20)
6,Loc(6),Loc(29)
7,Loc(7),Loc(31)
1,Loc(8),Loc(26)
17,Loc(9),Loc(39)


Or we could be more explicit if autop didn't work

In [5]:
def file_to_subj(loc):
    return loc.split('/')[-1].split('_')[0]

# Actually load it this time
data = data.add_data_files(files, file_to_subj)
data

,timeseries_lh,timeseries_rh
13,Loc(0),Loc(36)
9,Loc(1),Loc(27)
8,Loc(2),Loc(37)
2,Loc(3),Loc(35)
16,Loc(4),Loc(38)
11,Loc(5),Loc(20)
6,Loc(6),Loc(29)
7,Loc(7),Loc(31)
1,Loc(8),Loc(26)
17,Loc(9),Loc(39)


What's this though? Why are the files showing up as Loc(int). Whats going on is that the data files are really stored as just integers, see:

In [6]:
data['timeseries_lh']

13     0.0
9      1.0
8      2.0
2      3.0
16     4.0
11     5.0
6      6.0
7      7.0
1      8.0
17     9.0
19    10.0
15    11.0
10    12.0
3     13.0
14    14.0
0     15.0
18    16.0
5     17.0
4     18.0
12    19.0
Name: timeseries_lh, dtype: float64

They correspond to locations in a stored file mapping (note: you don't need to worry about any of this most of the time)

In [7]:
data.file_mapping[0], data.file_mapping[1], data.file_mapping[2]  

(DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/13_lh.npy'),
 DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/9_lh.npy'),
 DataFile(loc='/home/sage/BPt/Examples/Short_Examples/fake_time_data/8_lh.npy'))

Let's add a fake target to our dataset now

In [8]:
data['t'] = np.random.random(len(data))
data.set_target('t', inplace=True)
data

,timeseries_lh,timeseries_rh
13,Loc(0),Loc(36)
9,Loc(1),Loc(27)
8,Loc(2),Loc(37)
2,Loc(3),Loc(35)
16,Loc(4),Loc(38)
11,Loc(5),Loc(20)
6,Loc(6),Loc(29)
7,Loc(7),Loc(31)
1,Loc(8),Loc(26)
17,Loc(9),Loc(39)


Next we will generate a Loader to apply a parcellation, then extract a measure of connectivity.

In [9]:
from neurotools.transform import SurfLabels

lh_parc = SurfLabels(labels='data/lh.aparc.annot', vectorize=False)
rh_parc = SurfLabels(labels='data/rh.aparc.annot', vectorize=False)

We can see how this object works on example data first.

In [10]:
ex_lh = data.file_mapping[0].load()
ex_lh.shape

(100, 10242)

In [11]:
trans = lh_parc.fit_transform(ex_lh)
trans.shape

(100, 35)

We essentially get a reduction from 10242 features to 35.

Next, we want to transform the matrix into a correlation matrix.

In [12]:
from BPt.extensions import SingleConnectivityMeasure
scm = SingleConnectivityMeasure(kind='covariance', discard_diagonal=True, vectorize=True)

In [13]:
scm.fit_transform(trans).shape

(595,)

The single connectivity measure is just a wrapper designed to let the ConnectivityMeasure from nilearn work with a single subject's data at a time.

Next, let's use the input special Pipe wrapper to compose these two objects into their own pipeline

In [14]:
lh_loader = bp.Loader(bp.Pipe([lh_parc, scm]), scope='_lh')
rh_loader = bp.Loader(bp.Pipe([rh_parc, scm]), scope='_rh')

Define a simple pipeline with just our loader steps, then evaluate with mostly default settings.

In [15]:
pipeline = bp.Pipeline([lh_loader, rh_loader, bp.Model('linear')])

results = bp.evaluate(pipeline, data)
results

Predicting target = t
Using problem_type = regression
Using scope = all (defining a total of 2 features).
Evaluating 20 total data points.


Folds:   0%|          | 0/5 [00:00<?, ?it/s]


Training Set: (16, 2)
Validation Set: (4, 2)
Fit fold in 1.8 seconds.
explained_variance: 0.0289
neg_mean_squared_error: -0.0992

Training Set: (16, 2)
Validation Set: (4, 2)
Fit fold in 1.7 seconds.
explained_variance: -0.4599
neg_mean_squared_error: -0.0716

Training Set: (16, 2)
Validation Set: (4, 2)
Fit fold in 1.2 seconds.
explained_variance: -0.2655
neg_mean_squared_error: -0.0564

Training Set: (16, 2)
Validation Set: (4, 2)
Fit fold in 0.8 seconds.
explained_variance: -0.8946
neg_mean_squared_error: -0.0325

Training Set: (16, 2)
Validation Set: (4, 2)
Fit fold in 2.0 seconds.
explained_variance: 0.3824
neg_mean_squared_error: -0.0405



EvalResults
------------
explained_variance: -0.2417 ± 0.4330
neg_mean_squared_error: -0.0600 ± 0.0237

Saved Attributes: ['estimators', 'preds', 'timing', 'estimator', 'train_subjects', 'val_subjects', 'feat_names', 'ps', 'mean_scores', 'std_scores', 'weighted_mean_scores', 'scores', 'fis_', 'coef_', 'cv']

Available Methods: ['to_pickle', 'compare', 'get_X_transform_df', 'get_inverse_fis', 'run_permutation_test', 'get_preds_dfs', 'subset_by', 'get_fis', 'get_coef_', 'permutation_importance']

Evaluated With:
target: t
problem_type: regression
scope: all
subjects: all
random_state: 1


Don't be discouraged that this didn't work, we are after all trying to predict random noise with random noise ... 

In [16]:
# These are the steps of the pipeline
fold0_pipeline = results.estimators[0]
for step in fold0_pipeline.steps:
    print(step[0])

loader_pipe0
loader_pipe1
linear regressor


We can investigate pieces, or use special functions like

In [17]:
results.get_X_transform_df(data, fold=0)

,timeseries_rh_0,timeseries_rh_1,timeseries_rh_2,timeseries_rh_3,timeseries_rh_4,timeseries_rh_5,timeseries_rh_6,timeseries_rh_7,timeseries_rh_8,timeseries_rh_9,...,timeseries_lh_585,timeseries_lh_586,timeseries_lh_587,timeseries_lh_588,timeseries_lh_589,timeseries_lh_590,timeseries_lh_591,timeseries_lh_592,timeseries_lh_593,timeseries_lh_594
0,-0.000089,-0.000046,-0.000017,0.000043,0.000002,-0.000013,0.000050,-0.000036,-0.000055,-0.000024,...,9.389005e-06,0.000122,-0.000015,0.000007,1.921520e-05,-1.646211e-05,0.000015,-0.000019,0.000088,0.000003
1,0.000038,-0.000016,-0.000029,-0.000010,0.000021,0.000033,0.000003,0.000014,-0.000014,-0.000036,...,-8.621476e-06,0.000046,0.000030,0.000002,-1.902180e-05,1.930467e-05,-0.000001,-0.000090,0.000030,0.000009
10,0.000080,0.000059,-0.000005,-0.000020,0.000050,0.000014,-0.000030,-0.000002,-0.000015,0.000010,...,2.462981e-05,0.000056,0.000001,0.000016,1.151054e-05,3.873892e-06,0.000002,0.000018,0.000070,0.000004
12,0.000005,-0.000004,-0.000046,0.000043,-0.000084,-0.000019,0.000108,-0.000128,-0.000079,0.000058,...,2.602420e-06,0.000034,0.000003,-0.000007,-1.585256e-05,9.441104e-06,-0.000005,0.000063,0.000012,0.000002
13,0.000104,0.000013,0.000012,-0.000092,-0.000005,0.000011,0.000024,0.000070,0.000063,0.000037,...,6.666113e-07,-0.000028,-0.000018,0.000004,2.614790e-05,1.830400e-06,-0.000010,-0.000105,-0.000020,0.000087
15,0.000008,-0.000019,-0.000037,-0.000110,-0.000072,0.000013,0.000010,0.000072,-0.000024,-0.000018,...,1.319442e-05,-0.000002,-0.000015,0.000018,-7.213823e-06,1.180117e-05,-0.000011,0.000041,-0.000028,-0.000015
16,-0.000149,-0.000041,0.000100,0.000018,0.000067,0.000012,0.000035,-0.000087,0.000010,-0.000039,...,-1.011534e-05,-0.000039,0.000008,0.000008,-6.898275e-06,-3.471210e-05,-0.000004,-0.000016,0.000046,-0.000027
17,-0.000035,-0.000019,0.000004,0.000062,-0.000072,-0.000021,0.000016,0.000039,-0.000022,0.000022,...,2.189124e-05,0.000011,-0.000004,0.000028,5.646921e-06,-9.971563e-06,-0.000013,0.000121,0.000035,0.000048
19,0.000070,-0.000054,-0.000099,-0.000031,-0.000005,0.000018,0.000074,0.000035,0.000063,-0.000019,...,9.041580e-06,-0.000031,0.000009,-0.000028,2.153300e-05,-2.700178e-06,0.000003,-0.000042,0.000050,-0.000018
2,-0.000080,-0.000014,-0.000077,0.000005,-0.000033,0.000004,-0.000010,0.000001,-0.000007,0.000006,...,1.552383e-05,0.000008,0.000012,0.000017,1.366919e-05,-1.173042e-07,0.000015,-0.000097,-0.000046,-0.000006
